In [91]:
import pandas as pd
import numpy as np 
from tabulate import tabulate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import metrics
nltk.download('vader_lexicon')
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/prashantkarna/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [92]:
import pandas as pd
data = pd.read_csv('/Users/prashantkarna/Documents/Research Materials/Datasets/depression_dataset_reddit_cleaned.csv')
# rename the columns
data = data.rename(columns={'clean_text': 'text', 'is_depression': 'class'})



In [93]:
data.shape

(7731, 2)

In [94]:

# import numpy as np
# data_split = np.array_split(data, 3)
# data = data_split[0]
# data = data.drop('Unnamed: 0',axis=1)

In [95]:
data.head(10)

,text,class
0,we understand that most people who reply immed...,1
1,welcome to r depression s check in post a plac...,1
2,anyone else instead of sleeping more when depr...,1
3,i ve kind of stuffed around a lot in my life d...,1
4,sleep is my greatest and most comforting escap...,1
5,i m year old turning soon in a few month i liv...,1
6,i live alone and despite me being prone to lon...,1
7,i m not looking for sympathy just simply to st...,1
8,i don t know how to communicate all of my thou...,1
9,mom i m sad it hurt in my heart the feeling fa...,1


In [96]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prashantkarna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [100]:
X = data.drop('class', axis=1)
y = data['class']


In [101]:
# To remove emails
email_regex = r'([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
regexes_to_remove = [email_regex, r'Subject:', r'Re:']

for i in range(0, len(X)):
    # removing all special charachter
    review = re.sub('[^a-zA-Z]', ' ', str(X['text'][i]))
    # make document as lowerCase
    review = review.lower()
    # splitting the documents into words for ex ['iam', 'omar']
    review = review.split()
    # make limmatization --> (change, changing, changes)---> (change)
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    # join the document agian
    review = ' '.join(review)
    
    # removing mails
    for r in regexes_to_remove:
        X['text'][i] = re.sub(r, '', review)


In [102]:
# Count the number of instances in each class
class_counts = data['class'].value_counts()

# Print the class counts
print(class_counts)

0    3900
1    3831
Name: class, dtype: int64


In [ ]:
# from sklearn.utils import resample
# # balance the dataset
# df_majority = data[data['class'] == '0']
# df_minority = data[data['class'] == '1']
# df_minority_upsampled = resample(df_minority, replace=True, n_samples=len(df_majority), random_state=123)
# data = pd.concat([df_majority, df_minority_upsampled])

In [103]:
# Count the number of instances in each class
class_counts = data['class'].value_counts()

# Print the class counts
print(class_counts)

0    3900
1    3831
Name: class, dtype: int64


In [104]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
tfidf_vectorizer_uni = TfidfVectorizer(max_features=10000,ngram_range=(1,1))
tfidf_vectorizer_bi = TfidfVectorizer(max_features=10000, ngram_range=(1,2))
tfidf_vectorizer_tri = TfidfVectorizer(max_features=10000, ngram_range=(1,3))

X_tfidf = tfidf_vectorizer.fit_transform(X['text'])
# X_tfidf_test = tfidf_vectorizer.transform(X['text'])

X_tfidf_uni = tfidf_vectorizer_uni.fit_transform(X['text'])
# X_tfidf_test_uni = tfidf_vectorizer_uni.transform(X['text'])

X_tfidf_bi = tfidf_vectorizer_bi.fit_transform(X['text'])
# X_tfidf_test_bi =tfidf_vectorizer_bi.transform(X'text'])

X_tfidf_tri = tfidf_vectorizer_tri.fit_transform(X['text'])
# X_tfidf_test_tri = tfidf_vectorizer_tri.transform(X_test['text'])

In [105]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

# Feature selection using ExtraTreesClassifier and SelectFromModel

selector = SelectFromModel(ExtraTreesClassifier(n_estimators=300, random_state=42))

                                        
# Unigram feature selection
selector_tfidf= selector.fit(X_tfidf, y)
X_selection = selector_tfidf.transform(X_tfidf)


# Unigram feature selection
selector_uni= selector.fit(X_tfidf_uni, y)
X_selection_uni = selector_uni.transform(X_tfidf_uni)

# Bigram feature selection
selector_bi = selector.fit(X_tfidf_bi, y)
X_selection_bi = selector_bi.transform(X_tfidf_bi)

# Trigram feature selection
selector_tri= selector.fit(X_tfidf_tri, y)
X_selection_tri = selector_tri.transform(X_tfidf_tri)

In [ ]:
# print(X_selection)

In [ ]:
# # extracted features to csv
# X_df = pd.DataFrame(X_selection.todense())
# X_df.to_csv('/Users/prashantkarna/Documents/Research Materials/final code/feature_dataset.csv', index=False)

In [ ]:
# # extracted features to cs
# X_df_uni = pd.DataFrame(X_selection.todense())
# X_df_uni.to_csv('/Users/prashantkarna/Documents/Research Materials/final code/feature_uni_dataset.csv', index=False)

In [ ]:
# # extracted features to csv
# X_df_bi = pd.DataFrame(X_selection.todense())
# X_df_bi.to_csv('/Users/prashantkarna/Documents/Research Materials/final code/feature_bi_dataset.csv', index=False)

In [ ]:
# # extracted features to csv
# X_df_tri = pd.DataFrame(X_selection.todense())
# X_df_tri.to_csv('/Users/prashantkarna/Documents/Research Materials/final code/feature_tri_dataset.csv', index=False)

In [ ]:
# data1 = pd.read_csv('/Users/prashantkarna/Documents/Research Materials/final code/feature_tri_dataset.csv')
# data1.columns

# # X = data1.drop('class', axis=1)
# # y = data1['class']

In [106]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# Set up Stratified KFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define classifiers
svm = LinearSVC()
lr = LogisticRegression(C=500, penalty='l1', solver='saga')
nb = MultinomialNB(alpha=0.01)

# Define lists to store accuracy scores
svm_acc = []
lr_acc = []
nb_acc = []

# Perform stratified k-fold cross-validation on selected features
for train_index, test_index in skf.split(X_selection_bi, y):
    # Split data into training and testing sets
    X_train, X_test = X_selection_bi[train_index], X_selection_bi[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit and predict using LinearSVC
    svm.fit(X_train, y_train)
    svm_pred = svm.predict(X_test)
    svm_acc.append(accuracy_score(y_test, svm_pred))
    print("LinearSVC classification report:\n", classification_report(y_test, svm_pred))
    
    # Fit and predict using Logistic Regression
    lr.fit(X_train, y_train)
    lr_pred = lr.predict(X_test)
    lr_acc.append(accuracy_score(y_test, lr_pred))
    print("Logistic Regression classification report:\n", classification_report(y_test, lr_pred))
    
    # Fit and predict using Naive Bayes
    nb.fit(X_train, y_train)
    nb_pred = nb.predict(X_test)
    nb_acc.append(accuracy_score(y_test, nb_pred))
    print("Naive Bayes classification report:\n", classification_report(y_test, nb_pred))

# # Print best accuracy score for each classifier
# print("LinearSVC best accuracy:", max(svm_acc))
# print("Logistic Regression best accuracy:", max(lr_acc))
# print("Naive Bayes best accuracy:", max(nb_acc))
# # print("XGBoost best accuracy:", max(xgb_acc))



LinearSVC classification report:
               precision    recall  f1-score   support

           0       0.93      0.98      0.95       780
           1       0.98      0.93      0.95       767

    accuracy                           0.95      1547
   macro avg       0.95      0.95      0.95      1547
weighted avg       0.95      0.95      0.95      1547

Logistic Regression classification report:
               precision    recall  f1-score   support

           0       0.93      0.96      0.95       780
           1       0.96      0.93      0.95       767

    accuracy                           0.95      1547
   macro avg       0.95      0.95      0.95      1547
weighted avg       0.95      0.95      0.95      1547

Naive Bayes classification report:
               precision    recall  f1-score   support

           0       0.97      0.84      0.90       780
           1       0.86      0.97      0.91       767

    accuracy                           0.91      1547
   macro avg  

In [107]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Set up Stratified KFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define classifiers
svm = LinearSVC(C=1.0,dual=False, max_iter=1000,random_state=42)
lr = LogisticRegression()
nb = MultinomialNB()

# Define lists to store accuracy scores
svm_acc = []
lr_acc = []
nb_acc = []

# Perform stratified k-fold cross-validation on selected features
for train_index, test_index in skf.split(X_selection_tri, y):
    # Split data into training and testing sets
    X_train, X_test = X_selection_tri[train_index], X_selection_tri[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit and predict using LinearSVC
    svm.fit(X_train, y_train)
    svm_pred = svm.predict(X_test)
    svm_acc.append(accuracy_score(y_test, svm_pred))
    print("LinearSVC classification report:\n", classification_report(y_test, svm_pred))
    
    # Fit and predict using Logistic Regression
    lr.fit(X_train, y_train)
    lr_pred = lr.predict(X_test)
    lr_acc.append(accuracy_score(y_test, lr_pred))
    print("Logistic Regression classification report:\n", classification_report(y_test, lr_pred))
    
    # Fit and predict using Naive Bayes
    nb.fit(X_train, y_train)
    nb_pred = nb.predict(X_test)
    nb_acc.append(accuracy_score(y_test, nb_pred))
    print("Naive Bayes classification report:\n", classification_report(y_test, nb_pred))

# Print best accuracy score for each classifier
print("LinearSVC best accuracy:", max(svm_acc))
print("Logistic Regression best accuracy:", max(lr_acc))
print("Naive Bayes best accuracy:", max(nb_acc))



LinearSVC classification report:
               precision    recall  f1-score   support

           0       0.93      0.98      0.96       780
           1       0.98      0.93      0.95       767

    accuracy                           0.95      1547
   macro avg       0.96      0.95      0.95      1547
weighted avg       0.96      0.95      0.95      1547

Logistic Regression classification report:
               precision    recall  f1-score   support

           0       0.93      0.98      0.95       780
           1       0.98      0.92      0.95       767

    accuracy                           0.95      1547
   macro avg       0.95      0.95      0.95      1547
weighted avg       0.95      0.95      0.95      1547

Naive Bayes classification report:
               precision    recall  f1-score   support

           0       0.93      0.86      0.90       780
           1       0.87      0.94      0.90       767

    accuracy                           0.90      1547
   macro avg  

In [108]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Set up Stratified KFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define classifiers
svm = LinearSVC()
lr = LogisticRegression()
nb = MultinomialNB()

# Define lists to store accuracy scores
svm_acc = []
lr_acc = []
nb_acc = []

# Perform stratified k-fold cross-validation on selected features
for train_index, test_index in skf.split(X_selection_uni, y):
    # Split data into training and testing sets
    X_train, X_test = X_selection_uni[train_index], X_selection_uni[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit and predict using LinearSVC
    svm.fit(X_train, y_train)
    svm_pred = svm.predict(X_test)
    svm_acc.append(accuracy_score(y_test, svm_pred))
    print("LinearSVC classification report:\n", classification_report(y_test, svm_pred))
    
    # Fit and predict using Logistic Regression
    lr.fit(X_train, y_train)
    lr_pred = lr.predict(X_test)
    lr_acc.append(accuracy_score(y_test, lr_pred))
    print("Logistic Regression classification report:\n", classification_report(y_test, lr_pred))
    
    # Fit and predict using Naive Bayes
    nb.fit(X_train, y_train)
    nb_pred = nb.predict(X_test)
    nb_acc.append(accuracy_score(y_test, nb_pred))
    print("Naive Bayes classification report:\n", classification_report(y_test, nb_pred))

# Print best accuracy score for each classifier
print("LinearSVC best accuracy:", max(svm_acc))
print("Logistic Regression best accuracy:", max(lr_acc))
print("Naive Bayes best accuracy:", max(nb_acc))


LinearSVC classification report:
               precision    recall  f1-score   support

           0       0.93      0.97      0.95       780
           1       0.97      0.93      0.95       767

    accuracy                           0.95      1547
   macro avg       0.95      0.95      0.95      1547
weighted avg       0.95      0.95      0.95      1547

Logistic Regression classification report:
               precision    recall  f1-score   support

           0       0.92      0.98      0.95       780
           1       0.98      0.91      0.94       767

    accuracy                           0.95      1547
   macro avg       0.95      0.95      0.95      1547
weighted avg       0.95      0.95      0.95      1547

Naive Bayes classification report:
               precision    recall  f1-score   support

           0       0.93      0.85      0.89       780
           1       0.86      0.94      0.90       767

    accuracy                           0.89      1547
   macro avg  

In [109]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Set up Stratified KFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Define list to store accuracy scores
rf_acc = []

# Perform stratified k-fold cross-validation on selected features
for train_index, test_index in skf.split(X_selection_tri, y):
    # Split data into training and testing sets
    X_train, X_test = X_selection_tri[train_index], X_selection_tri[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit and predict using Random Forest Classifier
    rf.fit(X_train, y_train)
    rf_pred = rf.predict(X_test)
    rf_acc.append(accuracy_score(y_test, rf_pred))

# Print best accuracy score for Random Forest Classifier
print("Random Forest Classifier best accuracy:", max(rf_acc))


Random Forest Classifier best accuracy: 0.964424320827943


In [110]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Set up Stratified KFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define XGBoost Classifier
xgb = XGBClassifier(random_state=42)

# Define list to store accuracy scores
xgb_acc = []

# Perform stratified k-fold cross-validation on selected features
for train_index, test_index in skf.split(X_selection_tri, y):
    # Split data into training and testing sets
    X_train, X_test = X_selection_tri[train_index], X_selection_tri[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit and predict using XGBoost Classifier
    xgb.fit(X_train, y_train, eval_metric='error')
    xgb_pred = xgb.predict(X_test)
    xgb_acc.append(accuracy_score(y_test, xgb_pred))

# Print best accuracy score for XGBoost Classifier
print("XGBoost Classifier best accuracy:", max(xgb_acc))


XGBoost Classifier best accuracy: 0.9702457956015524
